In [59]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
import scipy as sp
%matplotlib ipympl

plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=200, formatter={'float': '{:.6f}'.format})
# np.set_printoptions(linewidth=200, formatter={'float': '{:.3e}'.format})

In [60]:
GHz = 1e9
fF = 1e-15
nH = 1e-9

In [72]:
def fluxonium(LF, CF, EJ, nmax):
    loop = sq.Loop(0.5)
    C_01   = sq.Capacitor(CF, 'fF')
    L_01   = sq.Inductor (LF, 'nH', loops=[loop])
    EJ_01  = sq.Junction(EJ, 'GHz', loops=[loop])
    elements = {(0, 1): [L_01, C_01, EJ_01]}
    circuit = sq.Circuit(elements)
    circuit.set_trunc_nums([nmax])
    return circuit

def resonator(LR, CR, nmax):
    L_01   = sq.Inductor (LR, 'nH')
    C_01   = sq.Capacitor(CR, 'fF')
    elements = {(0, 1): [L_01, C_01],}
    circuit = sq.Circuit(elements)
    circuit.set_trunc_nums([nmax])
    return circuit

def coupled_unit_cells(LF1, CF1, EJ1, LR1, CR1, LF2, CF2, EJ2, LR2, CR2, CC, nmax):
    # This ignores the internal flux*flux coupling, that is for Δ=0. 
    loop1 = sq.Loop(0.5)
    C_01   = sq.Capacitor(CF1, 'fF')
    L_01   = sq.Inductor (LF1, 'nH', loops=[loop1])
    EJ_01  = sq.Junction (EJ1, 'GHz',loops=[loop1])

    L_02   = sq.Inductor (LR1, 'nH')
    C_02   = sq.Capacitor(CR1, 'fF')
    
    loop2 = sq.Loop(0.5)
    C_03   = sq.Capacitor(CF2, 'fF')
    L_03   = sq.Inductor (LF2, 'nH', loops=[loop2])
    EJ_03  = sq.Junction (EJ2, 'GHz',loops=[loop2])

    L_04   = sq.Inductor (LR2, 'nH')
    C_04   = sq.Capacitor(CR2, 'fF')
    
    elements = {(0, 1): [L_01, C_01, EJ_01],
                (0, 2): [L_02, C_02],
                (0, 3): [L_03, C_03, EJ_03],
                (0, 4): [L_04, C_04],
                (1, 3): [CC],
                (1, 4): [CC],
                (2, 3): [CC],
                (2, 4): [CC]}
    
    circuit = sq.Circuit(elements)
    circuit.set_trunc_nums([nmax**2,nmax**2])
    
    return circuit

# def coupled_fluxoniums(L1, C1, EJ1, L2, C2, EJ2, CC):
#     loop1 = sq.Loop(0.5)
#     loop2 = sq.Loop(0.5)
#     
#     C_01 = sq.Capacitor(C1, 'fF')
#     L_01 = sq.Inductor (L1, 'nH', loops=[loop1])
#     EJ_01  = sq.Junction(EJ1, 'GHz', loops=[loop1])
#     
#     C_02 = sq.Capacitor(C2, 'fF')
#     L_02 = sq.Inductor (L2, 'nH', loops=[loop2])
#     EJ_02  = sq.Junction(EJ2, 'GHz', loops=[loop2])
#     
#     C_12 = sq.Capacitor(CC, 'fF')
#     
#     elements = {(0, 1): [L_01, C_01, EJ_01],
#                 (0, 2): [L_02, C_02, EJ_02],
#                 (1, 2): [C_12], }
#     return sq.Circuit(elements)

In [73]:
n_eig = 5
nmax = 15

In [77]:
LF1 = 3
CF1 = 3
EJ1 = 5
LR1 = 2
CR1 = 3

LF2 = 3
CF2 = 3
EJ2 = 5
LR2 = 2
CR2 = 3

CC = 2

only_inner = True
ompensate_extra_cap = True

In [78]:
H_unit_cell_1_full = coupled_unit_cells(LF1, CF1, EJ1, LR1, CR1, LF2, CF2, EJ2, LR2, CR2, CC, nmax).hamiltonian()

In [80]:
C_mat = sq_ext.C_mat_qubit_C_qubit(CC, CR=CR1, CF=CF1, CR_prime=CR2, CF_prime=CF2, only_inner=only_inner, compensate_extra_cap=ompensate_extra_cap, only_renormalization=False)

In [81]:
C_mat

array([[4.000000, 0.000000, 0.500000, 0.500000],
       [0.000000, 4.000000, -0.500000, -0.500000],
       [0.500000, -0.500000, 4.000000, 0.000000],
       [0.500000, -0.500000, 0.000000, 4.000000]])

In [55]:
H_F_1 = fluxonium(LF_1, CF_1, EJ_1, nmax).hamiltonian()
H_R_1 = resonator(LR_1, CR_1, nmax).hamiltonian()
I = qt.identity(H_F_1.shape[0])

H_unit_cell_1_comp = qt.tensor(H_F_1,I) + qt.tensor(I,H_R_1)

In [56]:
E_F_1= sq_ext.diag(H_F_1, n_eig, remove_ground=True)[0]
E_R_1= sq_ext.diag(H_R_1, n_eig, remove_ground=True)[0]
E_F_1, E_R_1

(array([0.000000, 51.123722, 102.734757, 154.743545, 207.076328]),
 array([0.000000, 64.974733, 129.949467, 194.924200, 259.898934]))

In [57]:
E_unit_cell_1_full = sq_ext.diag(H_unit_cell_1_full, n_eig, remove_ground=True)[0]
E_unit_cell_1_comp = sq_ext.diag(H_unit_cell_1_comp, n_eig, remove_ground=True)[0]

In [58]:
E_unit_cell_1_full, E_unit_cell_1_comp

(array([0.000000, 51.123722, 102.734757, 154.743545, 207.076328]),
 array([0.000000, 51.123722, 64.974733, 102.734757, 116.098456]))

# Past

In [12]:
coupled_res_full = coupled_fluxoniums(L1=L1, C1=C1, EJ1=EJ1, L2=L2, C2=C2, EJ2=EJ2, CC=CC)
coupled_res_full .set_trunc_nums([trunc_num, trunc_num])
H_coupled_res_full = coupled_res_full.hamiltonian()

In [13]:
C_mat = np.array([[C1 + CC, -CC],
                  [-CC, C2 + CC]])
C_inv = np.linalg.inv(C_mat)
C1_tilde = C_inv[0,0]**-1
C2_tilde = C_inv[1,1]**-1

In [15]:
fluxonium_1 = fluxonium(L=L1, C=C1_tilde, EJ=EJ1)
fluxonium_2 = fluxonium(L=L2, C=C2_tilde, EJ=EJ2)
fluxonium_1.set_trunc_nums([trunc_num])
fluxonium_2.set_trunc_nums([trunc_num])

In [17]:
Q_1 = fluxonium_1.charge_op(0)
Q_2 = fluxonium_2.charge_op(0)
Φ_1 = fluxonium_1.flux_op(0)
Φ_2 = fluxonium_2.flux_op(0)

H_1 = fluxonium_1.hamiltonian()
# H_1 = 1/2 * C1_tilde**-1 * fF**-1 * Q_1**2 + 1/2 * L1**-1 * nH**-1 * Φ_1**2 
H_2 = fluxonium_2.hamiltonian()
# H_2 = 1/2 * C2_tilde**-1 * fF**-1 * Q_2**2 + 1/2 * L2**-1 * nH**-1 * Φ_2**2 

I = qt.identity(H_1.shape[0])
H_coupled_res_comp = qt.tensor(H_1,I) + qt.tensor(I,H_2) + C_inv[0,1]* fF**-1 * qt.tensor(Q_1,Q_2)

In [18]:
E_coupled_res_comp = sq_ext.diag(H_coupled_res_comp, n_eig, remove_ground=True)[0]
E_coupled_res_full = sq_ext.diag(H_coupled_res_full, n_eig, remove_ground=True)[0]

In [19]:
E_coupled_res_comp, E_coupled_res_full

(array([0.000000, 38.035963, 65.492617, 76.297202, 103.637949]),
 array([0.000000, 38.035963, 65.492617, 76.297202, 103.637949]))